<a href="https://colab.research.google.com/github/chandan110791/NLP/blob/main/X1_Knowledge_Graph_Lesk_Word_Sense_Disambiguation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from nltk.corpus import wordnet as wn
from nltk import wsd

In [8]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

**Word Sense Dimabiguation**

In [2]:
X = 'The die is cast'
Y = 'Roll the die to get a 6'
Z = 'What is dead may never die'

In [9]:
wn.synsets('die')

[Synset('die.n.01'),
 Synset('die.n.02'),
 Synset('die.n.03'),
 Synset('die.v.01'),
 Synset('die.v.02'),
 Synset('die.v.03'),
 Synset('fail.v.04'),
 Synset('die.v.05'),
 Synset('die.v.06'),
 Synset('die.v.07'),
 Synset('die.v.08'),
 Synset('die.v.09'),
 Synset('die.v.10'),
 Synset('die.v.11')]

In [10]:
wn.synsets('die',pos=wn.NOUN)

[Synset('die.n.01'), Synset('die.n.02'), Synset('die.n.03')]

In [12]:
for syn in wn.synsets('die',pos=wn.NOUN):
  print(syn.definition())

a small cube with 1 to 6 spots on the six faces; used in gambling to generate random numbers
a device used for shaping metal
a cutting tool that is fitted into a diestock and used for cutting male (external) screw threads on screws or bolts or pipes or rods


In [14]:
for syn in wn.synsets('die',pos=wn.VERB):
  print(syn.definition())

pass from physical life and lose all bodily attributes and functions necessary to sustain life
suffer or face the pain of death
be brought to or as if to the point of death by an intense emotion such as embarrassment, amusement, or shame
stop operating or functioning
feel indifferent towards
languish as with love or desire
cut or shape with a die
to be on base at the end of an inning, of a player
lose sparkle or bouquet
disappear or come to an end
suffer spiritual death; be damned (in the religious sense)


Word Sense Disambiguation with Lesk Algorithm

In [15]:
print(X)

The die is cast


In [16]:
wsd.lesk(X.split(),'die')

Synset('die.v.07')

In [17]:
_.definition()

'cut or shape with a die'

In [18]:
wsd.lesk(X.split(),'die',pos=wn.NOUN).definition()

'a cutting tool that is fitted into a diestock and used for cutting male (external) screw threads on screws or bolts or pipes or rods'

In [21]:
print(Y)
print(wsd.lesk(Y.split(),'die',pos=wn.NOUN).definition())
print(wsd.lesk(Y.split(),'die',pos=wn.VERB).definition())


Roll the die to get a 6
a small cube with 1 to 6 spots on the six faces; used in gambling to generate random numbers
to be on base at the end of an inning, of a player


In [22]:
print(Z)
print(wsd.lesk(Z.split(),'die',pos=wn.NOUN).definition())
print(wsd.lesk(Z.split(),'die',pos=wn.VERB).definition())

What is dead may never die
a cutting tool that is fitted into a diestock and used for cutting male (external) screw threads on screws or bolts or pipes or rods
cut or shape with a die


Automatic POS Tagging + Lesk with spaCy

In [23]:
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
from spacy.cli import download
from spacy import load 
nlp = load ('en_core_web_sm')

In [32]:
import warnings

POS_MAP = {
      'VERB':wn.VERB,
      'NOUN':wn.NOUN,
      'PROPN':wn.NOUN

}


def lesk(doc,word):
  found = False
  for token in doc:
    print(token.text)
    if token.text == word:
        word=token
        found = True
        break
  if not found:
      raise ValueError(f'Word \" does not appear in document {doc.text}')
  pos = POS_MAP.get(word.pos_,False)
  print(pos)
  if not pos:
      warnings.warn(f'POS tag for {word.text} not found in wordnet.Falling back to default lesk behaviour')
  args = [c.text for c in doc],word.text
  kwargs = dict(pos=pos)
  return wsd.lesk(*args,**kwargs)

In [26]:
doc = nlp('Roll the die to get a 6')

In [33]:
lesk(doc,'die')

Roll
the
die
n


Synset('die.n.01')

In [35]:
lesk(doc,'die').definition()

Roll
the
die
n


'a small cube with 1 to 6 spots on the six faces; used in gambling to generate random numbers'

In [37]:
lesk(nlp('i work at google'),'google').definition()

i
work
at
google
n


"a widely used search engine that uses text-matching techniques to find web pages that are important and relevant to a user's search"

In [39]:
lesk(nlp('i wil google it.'),'google').definition()

i
wil
google
n


"a widely used search engine that uses text-matching techniques to find web pages that are important and relevant to a user's search"